In [1]:
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import os
import sunpy.map
from aiapy.calibrate.prep import register as aiaprep  # sunpy.instr.aia.aiaprep was moved to aiapy
from sunpy.net import Fido, attrs as a
from astropy import units as u
import datetime
import glob

from sunpy.physics.differential_rotation import solar_rotate_coordinate
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
plt.rcParams.update({'figure.autolayout': True})
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import MaxNLocator
df = pd.read_excel('IRIS_AIA.xlsx' , sheet_name='test')

folder = '/Users/delbe/OneDrive - FHNW/aia_304/'
    
for label, row in df.iterrows():
    p1_x1 = row[1]
    p1_y1 = row[2]
    p2_x2 = row[3]
    p2_y2 = row[4]

    # time_axis = row[5].split("T")

    gradient = (p2_y2-p1_y1)/(p2_x2-p1_x1)
    origin = p1_y1 - gradient * p1_x1

    y = np.arange(min(p1_y1, p2_y2),max(p1_y1, p2_y2)+1)
    x = (y - origin) / gradient

    coords = (np.column_stack((x.round().astype(int),y.astype(int))))


    final_image = []
    list_dates = []
    list_of_skycoords_i = []
    list_of_pxcoords_i = []
    list_of_coords_i = []
    

    files = glob.glob(folder + row[0] + "/*")
    
                                     
    aia_1 = sunpy.map.Map(files[0])
    aia_map_1 = aiaprep(aia_1)
    
    skycoords = aia_map_1.pixel_to_world(coords[:,0] * u.pix, coords[:,1] * u.pix)


    for i in range(0,len(files)-1):
    # for i in range(0,100):

        if i%10 == 0:
            print(i, end=',')
        
        try:         
            aia_map = sunpy.map.Map(files[i])

        except: 
            continue

        aia_i = aiaprep(aia_map)
        skycoords_i = solar_rotate_coordinate(skycoords, time=(aia_i.date - aia_1.date).to(u.s))
        pxcoords_i = aia_i.world_to_pixel(skycoords_i)
        coords_i = [pxcoords_i.y.value.round().astype('int'), pxcoords_i.x.value.round().astype('int')]
        
        
        spilltet_date = aia_i.date.isot.split("T")
        list_dates.append(spilltet_date[1])
        
        # aia_data = aia_i.data
        # zi = np.clip(aia_data, 0, aia_data.max())
        # zi = zi**0.4  # gamma correction
        # zi = np.flipud(zi)
        # zi = np.flipud(aia_i.data*0.4)[coords_i[:,1], coords_i[:,0]]
        # print(zi)
        zi = np.flipud(aia_i.data**0.4)[tuple(coords_i)]
        
        np.save(f"../coords/{row[0]} _copy" , zi)
        
        # print(zi)

        if len(final_image) == 0: 
            final_image = zi 
        else:
            final_image = np.vstack([final_image, zi])


    fig, ax = plt.subplots(figsize=[30,10])      

    plt.title(aia_i.date.isot)

    # defining and sitting the number of ticks:
    nticks = 20 
    ax.xaxis.set_major_locator(MaxNLocator(nticks))
    ax.set_xticklabels(list_dates[::int(len(list_dates)/nticks)], rotation = 50, horizontalalignment="right")

    # fig, ax = plt.figure(figsize=[30,10]) 
    im = plt.imshow(final_image.transpose(),cmap = aia_i.cmap)
    plt.colorbar()

    plt.xlabel('Time[UT]')
    plt.ylabel('Position')
    
    np.save(f"../aia_times/{row[0]} _copy" , list_dates)
    np.save(f"../aia_data/{row[0]} _copy" , final_image) # save the data as numpy array ".npy" in the folder aia_data

    plt.show()      